In [38]:
from tqdm.auto import tqdm
import os
from torch.utils.data import DataLoader

In [39]:
%cd C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection

C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection


In [40]:
import torch
import time 
from tqdm.auto import tqdm
from faster_rcnn_od.utils.comman import custom_collate, Averager, SaveBestModel, save_loss_plot, save_model, CustDat
from collections import namedtuple

In [41]:
trainConfig = namedtuple("train_faster_rcnn", [
'root_dir','model_loader_path',
 'train_loader_path','valid_loader_path',
 'outputs', 'params_epoches', 'params_lr',
 'params_momentum', 'params_weight_decay', 'params_batch_size'
])

In [42]:
from faster_rcnn_od.constants import *
from faster_rcnn_od.utils.comman import read_yaml, create_directories, custom_collate, CustDat
from tqdm.auto import tqdm

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def train_config(self) -> trainConfig:
        config = self.config.train_faster_rcnn

        create_directories([config.root_dir])

        train_config = trainConfig (
            root_dir=config.root_dir,
            model_loader_path=config.model_loader_path,
            train_loader_path=config.train_loader_path,
            valid_loader_path= config.valid_loader_path,
            outputs= config.outputs,
            params_epoches=self.params.EPOCHS,
            params_lr=self.params.LEARNING_RATE,
            params_momentum=self.params.MOMENTUM,
            params_weight_decay = self.params.WEIGHT_DECAY,
            params_batch_size = self.params.BATCH_SIZE
        )

        return train_config

In [44]:
class training:
    
    global train_itr
    global train_loss_list
    DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    def __init__(self, config: trainConfig):
        
        self.config = config
        print(self.config)
        
    def base_model(self):
        return torch.load(self.config.model_loader_path)
    
    def train_start(self):
        
        # print(f"Number of training samples: {len(train_dataset)}")
        # print(f"Number of validation samples: {len(valid_dataset)}\n")
        # initialize the model and move to the computation device
        model = self.base_model()
        model = model.to(self.DEVICE)
        # get the model parameters
        params = [p for p in model.parameters() if p.requires_grad]
        # define the optimizer
        optimizer = torch.optim.SGD(params, lr=self.config.params_lr, momentum=self.config.params_momentum, weight_decay=self.config.params_weight_decay)
        # initialize the Averager class
        train_loss_hist = Averager()
        val_loss_hist = Averager()
        self.train_itr = 1
        self.val_itr = 1
        # train and validation loss lists to store loss values of all...
        # ... iterations till ena and plot graphs for all iterations
        # name to save the trained model with
        MODEL_NAME = 'model'
        # whether to show transformed images from data loader or not
        save_best_model = SaveBestModel()
        # start the training epochs
        NUM_EPOCHS = 1
        for epoch in range(NUM_EPOCHS):
            print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
            # reset the training and validation loss histories for the current epoch
            train_loss_hist.reset()
            val_loss_hist.reset()
            # start timer and carry out training and validation
            start = time.time()
            train_loss, loss_train = self.train(self.train_data_loader('train_loder.pt'), model, optimizer)
            val_loss, loss_val = self.validate(self.train_data_loader('val_loader.pt'), model)
            train_loss_hist.send(loss_train)
            train_loss_hist.send(loss_val)
            print(f"Epoch #{epoch+1} train loss: {train_loss_hist.value:.3f}")   
            print(f"Epoch #{epoch+1} validation loss: {val_loss_hist.value:.3f}")   
            end = time.time()
            print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
            # save the best model till now if we have the least loss in the...
            # ... current epoch
            save_best_model(
                val_loss_hist.value, epoch, model, optimizer
            )
            # save the current epoch model
            save_model(epoch, model, optimizer)
            # save loss plot
            OUT_DIR =self.config.outputs
            save_loss_plot(OUT_DIR, train_loss, val_loss)
            
            # sleep for 5 seconds after each epoch
            time.sleep(5)
        
    def train_data_loader(self, file_name):  
        path = self.config.train_loader_path
        path_tr = os.path.join(path, file_name)
        data = torch.load(path_tr)
        for i in data:
            imgs = []
            targets = []
            for d in i:
                imgs.append(d[0])
                targ = {}
                targ['boxes'] = d[1]['boxes']#.to(device)
                targ['labels'] = d[1]['label']#.to(device)
                targets.append(targ)
        return (targets, imgs)
        
    def train(self,train_data_loader, model, optimizer):
        
        print('Training')
        global train_itr
        global train_loss_list
        train_loss_list = []
        
        # initialize tqdm progress bar
        prog_bar = tqdm(train_data_loader, total=len(train_data_loader))
        targets, images = train_data_loader
        for i, data in enumerate(prog_bar):
            
            optimizer.zero_grad()
            loss_dict = model(images, targets)
            print(loss_dict)
            losses = sum(loss for loss in loss_dict[0].values())
            loss_value = losses.item()
            train_loss_list.append(loss_value)
            train_loss_hist = Averager()
            train_loss_hist.send(loss_value)
            losses.backward()
            optimizer.step()
            self.train_itr += 1
            
        
            # update the loss value beside the progress bar for each iteration
            prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
        return train_loss_list, loss_value
    
    
    def validate(self, valid_data_loader, model ):
        print('Validating')
        global val_itr
        global val_loss_list
        val_loss_list = []
        
        # initialize tqdm progress bar
        prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
        
        for i, data in enumerate(prog_bar):
            targets, images  = valid_data_loader
            
            
            with torch.no_grad():
                
                loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict[0].values())
            loss_value = losses.item()
            val_loss_list.append(loss_value)
            
           # val_loss_hist.send(loss_value)
            self.val_itr += 1
            # update the loss value beside the progress bar for each iteration
            prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
        return val_loss_list, loss_value
     
        

In [45]:
try:
    config = ConfigurationManager()
    train_model_config = config.train_config()
    prepare_base_model = training(config=train_model_config)
    prepare_base_model.train_start()
except Exception as e:
    raise e

[2023-05-07 18:58:55,749: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-05-07 18:58:55,754: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-05-07 18:58:55,755: INFO: comman: created directory at: artifacts]
[2023-05-07 18:58:55,757: INFO: comman: created directory at: artifacts/trained_output_facts]
train_faster_rcnn(root_dir='artifacts/trained_output_facts', model_loader_path='artifacts/prepare_base_model/base_model.pt', train_loader_path='artifacts\\transformed_data', valid_loader_path='artifacts\\transformed_data\\val_loader.pt', outputs='artifacts/trained_output_facts', params_epoches=1, params_lr=0.01, params_momentum=0.9, params_weight_decay=0.0005, params_batch_size=4)


c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\torch\serialization.py:799: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"



EPOCH 1 of 1
Training


  0%|          | 0/2 [00:00<?, ?it/s]code/__torch__/torchvision/models/detection/faster_rcnn.py:103: UserWarning: RCNN always returns a (Losses, Detections) tuple in scripting


({'loss_classifier': tensor(1.9353, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.8701, grad_fn=<DivBackward1>), 'loss_objectness': tensor(2.5696, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.3510, grad_fn=<DivBackward1>)}, [])


...

({'loss_classifier': tensor(1.0300, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.8499, grad_fn=<DivBackward1>), 'loss_objectness': tensor(0.7337, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.1522, grad_fn=<DivBackward1>)}, [])


Loss: 2.7658: 100%|██████████| 2/2 [00:15<00:00,  7.95s/it]


Validating


Loss: 6.5485: 100%|██████████| 2/2 [00:05<00:00,  2.76s/it]


Epoch #1 train loss: 4.657
Epoch #1 validation loss: 0.000
Took 0.561 minutes for epoch 0

Best validation loss: 0

Saving best model for epoch: 1

SAVING PLOTS COMPLETE...
